In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

This project involves looking at the effect of various potentials on various lengths of graphene both in 1 and 2 dimensions. Since it is time consuming to generate a Hamiltonian for each potential and each length, we look at two ways of speeding up the calculations.
1. Generate the Hamiltonian for a large unit cell like 50 nm X 50 nm with no applied potential. If we then want to study a system of dimensions 20 nm X 10 nm, post process the TG_hr.dat of the large unit cell to remove all atoms and interactions above 20 nm in the x and 10 nm in the y.
2. Regardless of the type and value of applied potential, obtain the Hamiltonian in the absence of potential and simply add the right potential values along the diagonals as a post processing step rather than through wanniertools.

In [31]:
def load_reference_data(filename):
    df = pd.read_csv(filename, header=None, sep='\s+')
    df.columns = [
        'R_vec_x', 'R_vec_y', 'R_vec_z',
        'atom1_index', 'atom2_index',
        'atom1_x_cart', 'atom1_y_cart', 'atom1_z_cart',
        'atom1_x_frac', 'atom1_y_frac', 'atom1_z_frac',
        'hopping_real', 'hopping_imag'
    ]
    return df

def compare_dataframes_with_tolerance(df1, df2, tol=1e-4):
    if not df1.columns.equals(df2.columns):
        return False

    float_cols = df1.select_dtypes(include=[np.float64, np.float32]).columns
    for col in float_cols:
        if not np.allclose(df1[col], df2[col], atol=tol, rtol=0):
            return False

    non_float_cols = df1.select_dtypes(exclude=[np.float64, np.float32]).columns
    if not df1[non_float_cols].equals(df2[non_float_cols]):
        return False

    return True

def write_to_file(output_path,data):
    columns_to_write = ['R_vec_x', 'R_vec_y', 'R_vec_z', 'atom1_index', 'atom2_index', 'hopping_real', 'hopping_imag']

    data_to_write = data[columns_to_write]


    formatted_data = pd.DataFrame({
    'R_vec_x': data_to_write['R_vec_x'].astype(int).apply('{:>4}'.format),
    'R_vec_y': data_to_write['R_vec_y'].astype(int).apply('{:>4}'.format),
    'R_vec_z': data_to_write['R_vec_z'].astype(int).apply('{:>4}'.format),
    'atom1_index': data_to_write['atom1_index'].astype(int).apply('{:>4}'.format),
    'atom2_index': data_to_write['atom2_index'].astype(int).apply('{:>4}'.format),
    'hopping_real': data_to_write['hopping_real'].apply('{:>20.10E}'.format),
    'hopping_imag': data_to_write['hopping_imag'].apply('{:>20.10E}'.format)
})

    formatted_string = formatted_data.apply(lambda x: ' '.join(x), axis=1)

    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, 'TG_hr_modified.dat'), 'w') as file:
        file.write('\n'.join(formatted_string))
    print("File written successfully with specified format.")

In [48]:
origin_path = "graphene/cut_hamiltonian/origin_3by3/potential_0"
origin_file = f"{origin_path}/TG_hr_with_atom_coord.dat"
origin_data = load_reference_data(origin_file)

reference_path = "graphene/cut_hamiltonian/cut_2by2/Reference/potential_0"
reference_file = f"{reference_path}/TG_hr_with_atom_coord.dat"
reference_data = load_reference_data(reference_file)

In [60]:
x_cut = 7.9
y_cut = 4.9

valid_indices = origin_data[(origin_data['atom1_x_cart'] <= x_cut) & (origin_data['atom1_y_cart'] <= y_cut)]['atom1_index'].unique()
cut_data = origin_data[origin_data['atom1_index'].isin(valid_indices) & origin_data['atom2_index'].isin(valid_indices)]
cut_data = cut_data[['R_vec_x', 'R_vec_y', 'R_vec_z', 'atom1_index', 'atom2_index', 'hopping_real', 'hopping_imag']]
reference_data = reference_data[['R_vec_x', 'R_vec_y', 'R_vec_z', 'atom1_index', 'atom2_index', 'hopping_real', 'hopping_imag']]

compare_dataframes_with_tolerance(cut_data, reference_data, tol=1e-4)

False

In [55]:
output_path = "graphene/cut_hamiltonian/cut_2by2"
write_to_file(output_path,cut_data)

File written successfully with specified format.


In [56]:
cut_data["atom1_x_cart"].unique()

array([0.710141, 1.420281, 2.840563, 3.550704, 4.970986, 5.681127,
       7.101408, 7.811549])

In [57]:
cut_data["atom1_y_cart"].unique()

array([1.23, 3.69, 2.46, 0.  ])

In [58]:
reference_data["atom1_x_cart"].unique()

array([0.710141, 1.420281, 2.840563, 3.550704, 4.970986, 5.681127,
       7.101408, 7.811549])

In [59]:
reference_data["atom1_y_cart"].unique()

array([1.23, 3.69, 0.  , 2.46])